# Suspicious Scheduled Task Creation

**schtasks.exe creates tasks with SYSTEM / highest privileges / suspicious paths — common persistence.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **task name**, **author**, **action path**, **trigger**, **payload location**.


## Investigation Queries

Identify schtasks executions and enumerate scheduled tasks on the host.


In [ ]:
// KQL
DeviceProcessEvents
| where TimeGenerated > ago(30d)
| where DeviceName == '<HOST>'
| where FileName =~ 'schtasks.exe'
| where ProcessCommandLine has '/create'
| where ProcessCommandLine has_any ('/ru SYSTEM','/rl HIGHEST','\\Microsoft\\Windows\\')
| project TimeGenerated, AccountName, ProcessCommandLine, InitiatingProcessFileName


In [ ]:
# Splunk SPL
index=endpoint Image=*\\schtasks.exe CommandLine="*/create*" (CommandLine="*/ru SYSTEM*" OR CommandLine="*/rl HIGHEST*" OR CommandLine="*\\Microsoft\\Windows\\*")
| table _time host user CommandLine ParentImage


In [ ]:
# PowerShell
## Enumerate tasks on Windows host
schtasks /query /fo LIST /v

## Look for suspicious task names/actions
Get-ScheduledTask | Select TaskName, TaskPath, State


## Containment & Response

Disable/remove the task and delete payload. Hunt for the same task name across fleet.


In [ ]:
# PowerShell
## Disable a task
schtasks /Change /TN "<TASK_NAME>" /DISABLE

## Delete a task
schtasks /Delete /TN "<TASK_NAME>" /F


In [ ]:
# PowerShell
## Remove payload (example)
Remove-Item -Force "C:\Users\<USER>\AppData\Roaming\<payload>.ps1"
